# Step2: Image subtraction

Run this in v30 instead of v29. 

Use `subtractTask = lsst.ip.diffim.subtractImages.SimplifiedSubtractTask` to first run image subtraction for original images to get kernels.

Then rerun it for injected images with the kernels -- save the kernels for repeated use (for the visit-template image pair), when we inject different stamps. 

Note when we run image subtraction for the first time without the kernel, a source detection run happens, and it is different from the direct source detection of the visit image. 

The original source catalog (not source table) of dp1 is not published and can not be used by AL, which requires a source catalog.

Reference:

https://community.lsst.org/t/issues-with-image-subtraction/10429


In [ ]:
import lib.imdiff as di
import lib.tools as tl

## Original images

Load images.

In [ ]:
tract = 5063
patch = 14
band = 'i'

template_image = tl.load_pickle(f"template_{tract}_{patch}_{band}", tl.FIG_FOLDER)

In [ ]:
visit = 2024110800245
detector = 0

visit_image = tl.load_pickle(f"visit_{visit}_{detector}", tl.FIG_FOLDER)

Warp the template to match the visit image.

In [ ]:
warped_template = di.warp_image(visit_image, template_image)

Run image subtraction and save the kernel.

In [ ]:
difference_image, kernel = di.run_subtract_task(warped_template, visit_image)

In [ ]:
difference_image.writeFits("%s/%s.fits"%(tl.FIG_FOLDER, "diff_image0"))

In [ ]:
kernel_filename = "%s/%s.fits"%(tl.FIG_FOLDER, "kernel")
kernel.writeFits(kernel_filename)

In [ ]:
dia_sources = di.detect_dia_sources(visit_image, warped_template, difference_image)

In [ ]:
dia_sources.writeFits("%s/diaSources.fits"%tl.CATALOG_FOLDER)
dia_sources.asAstropy().write("%s/diaSources.csv"%tl.CATALOG_FOLDER, overwrite=True)

In [ ]:
dia_sources_tab_df = di.sourceCat2Tab(dia_sources, difference_image, difference_image.getFilter())
dia_sources_tab_df.to_csv("%s/diaSources_tab.csv"%tl.CATALOG_FOLDER, index=False)

## Injected images

In [ ]:
template_image = tl.load_pickle(f"template_{tract}_{patch}_{band}", tl.FIG_FOLDER)
visit_image = tl.load_pickle(f"visit_{visit}_{detector}", tl.FIG_FOLDER)

In [ ]:
warped_template = di.warp_image(visit_image, template_image)

In [ ]:
kernel = di.get_kernel(kernel_filename)
difference_image, _ = di.run_subtract_task(warped_template, visit_image, kernel)

In [ ]:
difference_image.writeFits("%s/%s.fits"%(tl.FIG_FOLDER, "diff_image"))

In [ ]:
dia_sources = di.detect_dia_sources(visit_image, warped_template, difference_image)

In [ ]:
dia_sources.writeFits("%s/injected_diaSources.fits"%tl.CATALOG_FOLDER)
dia_sources.asAstropy().write("%s/injected_diaSources.csv"%tl.CATALOG_FOLDER, overwrite=True)

In [ ]:
dia_sources_tab_df = di.sourceCat2Tab(dia_sources, difference_image, difference_image.getFilter())
dia_sources_tab_df.to_csv("%s/injected_diaSources_tab.csv"%tl.CATALOG_FOLDER, index=False)